# Predicates Requirement

To show that using predicates on input and output states of different independent modules in quantum circuits to verify correctness is useful, here is a series of versions of Grover's algorithm with different bugs in different modules, to show it is possible to violate these predicates and still have a program that compiles. These versions will compile, but the logic underlying is wrong. This is possible when a programmer writes a module. It would be difficult to tell where such an error has occurred since it is a logical error, not a syntactical error.

## Correct Grover's algorithm

In [20]:
import pennylane as qml
import numpy as np

dev = qml.device("default.qubit", wires=2)

def init_state_correct(wires):
    for w in wires :
        qml.Hadamard(w)

def oracle_correct(wires):
    # Flips the phase of |11> by 180 degrees (-1)
    qml.CZ(wires=wires)

def diffusion_correct(wires):
    for w in wires: qml.Hadamard(w)
    for w in wires: qml.PauliX(w)
    qml.CZ(wires=wires)
    for w in wires: qml.PauliX(w)
    for w in wires: qml.Hadamard(w)

@qml.qnode(dev)
def grover_correct():
    init_state_correct([0, 1])
    oracle_correct([0, 1])
    diffusion_correct([0, 1])
    return qml.probs(wires=[0, 1])

print(f"Probabilities: {grover_correct()}")
# Expected: [0, 0, 0, 1.0] -> 100% chance for |11>

Probabilities: [0. 0. 0. 1.]


This is the correct implementation of the Grover's algorithm.
Now we will see two different versions of this algorithm with possible errors that a programmer might introduce unknowingly

## Wrong Phase Gate

This code uses the wrong phase gate in the oracle module. It uses S gate instead of CZ gate. This is a one line error but the program still compiles.

In [21]:
def oracle_buggy(wires):
    qml.S(wires=0)

@qml.qnode(dev)
def circuit_bug_A():
    for w in range(2): qml.Hadamard(w)
    oracle_buggy([0, 1])
    diffusion_correct([0, 1])
    return qml.probs(wires=[0, 1])

print(f"Probabilities: {circuit_bug_A()}")
# Result: Probabilities are no longer concentrated on |11>.

Probabilities: [0.25 0.25 0.25 0.25]


If there is a RealAmplitudes() predicate check after the oracle module, this error would be caught immediately. 

The predicate InGroverSubspace() should include the RealAmplitudes() predicate in it

All built-in gates in PennyLane are unitary, so applying them to a normalised state will result in a normalised state. So a Normalised() predicate is only useful when a programmer is creating their own custom operator. 